Import Dependencies and Set Up Spark

In [ ]:
from pyspark.sql import SparkSession
from src.data_loader import load_data
from src.preprocessing import preprocess_data
from src.model_training import train_models
from src.evaluation import evaluate_model
from src.graphs import (
    plot_top_tfidf_words,
    plot_logreg_weights,
    plot_rf_feature_importance,
    plot_nb_info'rmative_words
)

# Start Spark session
spark = SparkSession.builder.appName("Sentiment140 Final Notebook").getOrCreate()


25/07/30 21:36:53 WARN Utils: Your hostname, codespaces-cceabc resolves to a loopback address: 127.0.0.1; using 10.0.1.10 instead (on interface eth0)
25/07/30 21:36:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/30 21:36:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.ExceptionInInitializerError
	at org.apache.spark.unsafe.array.ByteArrayMethods.<clinit>(ByteArrayMethods.java:52)
	at org.apache.spark.memory.MemoryManager.defaultPageSizeBytes$lzycompute(MemoryManager.scala:261)
	at org.apache.spark.memory.MemoryManager.defaultPageSizeBytes(MemoryManager.scala:251)
	at org.apache.spark.memory.MemoryManager.$anonfun$pageSizeBytes$1(MemoryManager.scala:270)
	at scala.runtime.java8.JFunction0$mcJ$sp.apply(JFunction0$mcJ$sp.java:23)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.memory.MemoryManager.<init>(MemoryManager.scala:270)
	at org.apache.spark.memory.UnifiedMemoryManager.<init>(UnifiedMemoryManager.scala:58)
	at org.apache.spark.memory.UnifiedMemoryManager$.apply(UnifiedMemoryManager.scala:207)
	at org.apache.spark.SparkEnv$.create(SparkEnv.scala:325)
	at org.apache.spark.SparkEnv$.createDriverEnv(SparkEnv.scala:196)
	at o

Load and Inspect the Data

In [ ]:
df = load_data("training.1600000.processed.noemoticon.csv", sample_fraction=0.1)
df.show(5, truncate=80)


Preprocess the Data


In [ ]:
processed_df, pipeline_model = preprocess_data(df)
processed_df.show(5, truncate=False)


Train the Models

In [ ]:
trained_models = train_models(processed_df)


 Evaluate Models with Accuracy, Precision, Recall, F1


In [ ]:
results = evaluate_models(trained_models)
for model_name, metrics in results.items():
    print(f"\n--- {model_name} ---")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")


Visualise Model Accuracy

In [ ]:
plot_model_comparison(results)


Interpretability Visualisations

In [ ]:
plot_top_tfidf_words(pipeline_model)
plot_logreg_coefficients(pipeline_model, trained_models["Logistic Regression"])
plot_rf_importances(pipeline_model, trained_models["Random Forest"])
plot_nb_word_differences(pipeline_model, trained_models["Naive Bayes"])
